## 1.この課題の目的
・公開されている実装を動かす際のノウハウを得る  
・論文で読んだ手法を大規模なデータに適用する  

【目的としないこと】  
・高い精度を出す  

## 2.進め方
・コードリーディング（1日目夜発表）  
・データセットを学習し推定（1日目〜2日目）  
・手法の説明を記述する。（2日目夜発表）  
sprint19も引き続き同じ題材を扱う。精度を改善するための手法を学び、実践する。  

## 3.U-Net
セグメンテーション手法としてU-Netを使う。

O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015

https://arxiv.org/pdf/1505.04597.pdf

実装は以下のものを利用する。  
https://github.com/zhixuhao/unet

### ■論文と実装の対応を見る
説明  
まず、U-Netにおいて重要だと考えた部分を列挙する。そして、それに対応するコードを見つけていく。

**【回答】**  
U-Netにおいて重要な部分は、ダウンサンプリングの出力特徴マップをアップサンプリング時に「keras.layers.concatenate」している箇所に尽きます。

「main.py」,　「model.py」, 「data.py」の３ファイル（zhixuhaoさんのGitHub）でTGS-salt（kaggle）のデータセットで学習を行いましたが、うまく学習出来ていませんでした。精度を改善するために、BatchNormalizationを畳込み後の結果に対して入れてみました。精度（loss）は入れる前と後では良くなっていましたが、predictの画像がやはりちゃんと想定した結果にはなりませんでした（単色画像となってしまいます）。

# 説明
## セグメンテーション
セグメンテーションとは何か。以下の内容を含めること。  
・セマンティックセグメンテーションとインスタンスセグメンテーション  
・物体検出や画像分類との違い  
・どのような手法があるか。  

**【回答】**  
・画像内の物体クラスとその輪郭まで正確に推定することをセマンティックセグメンテーション（semantic segmentation）と言います。ピクセル単位で推定します。画像を画素レベルで把握することです。  
インスタンスセグメンテーションは、バウンディングボックスで対象の物体を囲います。  
セマンティックセグメンテーションの方が上位の技術だと思われますが、必ずしもそういうわけでは無いようです。例えば物体の数を数えたい場合は、セマンティックの場合は重なった物体を区別出来なくなってしまうため、うまく数えることが出来ません。問題に合った手法を選択する必要があります。

・物体検出は、上記で回答した「セマンティックセグメンテーション」「インスタンスセグメンテーション」に大きく分けられます。つまり、物体検出とは物体が存在する領域を推定する手法のことを言います。   
画像分類（＝画像認識）は画像に写る内容を識別することです。  
「画像認識」は「物体認識」と「シーン認識」に分けられて、さらに「物体認識」は「インスタンス認識」と「クラス認識」に分けれらます。

・YOLO、Faster R-CNN、U-Netなどの手法があります。

## スキップコネクション
ディープラーニングにおけるスキップコネクションとは何か。以下の双方の手法に関して説明せよ。  
・U-Netのようなセグメンテーション手法  
・ResNetなどの画像分類手法  

**【回答】**  
ResNetはショートカットコネクションと論文に記載されていますが、手前にある層の出力を後ろにある層の入力とすることです。つまり、加算します。  
U-NetではKerasのconcatenate関数を利用しており、結合です。  
ResNetは加算、U-Netは結合が違いとなります。

main.py

In [ ]:
# main.py
from model import *
from data import *

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'/Users/tsuneo/kaggle/TGS_Salt/train/train','images','masks', data_gen_args, save_to_dir=None)

model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane_tomo.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene, steps_per_epoch=2, epochs=1,
                    callbacks=[model_checkpoint, EarlyStopping])

testGene = testGenerator("/Users/tsuneo/kaggle/TGS_Salt/test/images")
results = model.predict_generator(testGene,5,verbose=1)
saveResult("/Users/tsuneo/kaggle/TGS_Salt/test/images/", "/Users/tsuneo/kaggle/TGS_Salt/tomomasa/",
           results, flag_multi_class=True)


data.py

In [ ]:
# data.py
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

Sky = [128,128,30]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])


def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)



def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)



def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    import glob
    a = glob.glob("/Users/tsuneo/kaggle/TGS_Salt/test/images/*")
    #print("aaaaaaaaaaaaa;", a)
    #for i in range(num_image):
    for i in a:
        b = i.replace("/Users/tsuneo/kaggle/TGS_Salt/test/images/","")
        #print("-------b   :",b)
        #print("i:::::", i)
        #img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)
        img = io.imread(os.path.join(test_path, "{}".format(b)), as_gray=as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img
        #yield b


def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr


def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    #return img_out / 255
    return img_out



def saveResult(test_path,save_path,npyfile,flag_multi_class = False,num_class = 2):
    #print("b",b)
    #print("save_path         :", save_path)
    a = glob.glob(test_path,"*")
        
    for i,item in enumerate(npyfile):
        if num_class == 2:
            item = zero_and_one(item)

        np.save(os.path.join(save_path, "%d_predict" % i), item)
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        print("type(img),   img.shape", type(img), img.shape)
        print("-------------img-------------", img)
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)


def zero_and_one(item):
    """
    adding function 2018/12/12

    item: ndarray (height, width)
    only 2 class
    """
    item[item > 0.5] = 1
    item[item <= 0.5] = 0
    return item


model.py

In [ ]:
# model.py
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras import backend as keras

# 1ピクセル毎の２値分類
# ノード数＝チェンネル数、フィルタのサイズ
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    # add tomomasa -start-
    conv1 = BatchNormalization()(conv1)
    # add tomomasa -end-
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    # add tomomasa -start-
    conv1 = BatchNormalization()(conv1)
    # add tomomasa -end-
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)


    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    # add tomomasa -start-
    conv2 = BatchNormalization()(conv2)
    # add tomomasa -end-
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    # add tomomasa -start-
    conv2 = BatchNormalization()(conv2)
    # add tomomasa -end-
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)


    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    # add tomomasa -start-
    conv3 = BatchNormalization()(conv3)
    # add tomomasa -end-
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    # add tomomasa -start-
    conv3 = BatchNormalization()(conv3)
    # add tomomasa -end-
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)


    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    # add tomomasa -start-
    conv4 = BatchNormalization()(conv4)
    # add tomomasa -end-
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    # add tomomasa -start-
    conv4 = BatchNormalization()(conv4)
    # add tomomasa -end-
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)


    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    # add tomomasa -start-
    conv5 = BatchNormalization()(conv5)
    # add tomomasa -end-
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    # add tomomasa -start-
    conv5 = BatchNormalization()(conv5)
    # add tomomasa -end-
    drop5 = Dropout(0.5)(conv5)



    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    # add tomomasa -start-
    conv6 = BatchNormalization()(conv6)
    # add tomomasa -end-
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    # add tomomasa -start-
    conv6 = BatchNormalization()(conv6)
    # add tomomasa -end-

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    # add tomomasa -start-
    conv7 = BatchNormalization()(conv7)
    # add tomomasa -end-
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    # add tomomasa -start-
    conv7 = BatchNormalization()(conv7)
    # add tomomasa -end-
    # drop out 0.3

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    # add tomomasa -start-
    conv8 = BatchNormalization()(conv8)
    # add tomomasa -end-
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    # add tomomasa -start-
    conv8 = BatchNormalization()(conv8)
    # add tomomasa -end-

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    # add tomomasa -start-
    conv9 = BatchNormalization()(conv9)
    # add tomomasa -end-
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    # add tomomasa -start-
    conv9 = BatchNormalization()(conv9)
    # add tomomasa -end-
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    # ノード数＝チェンネル数、フィルタのサイズ
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model